# RuleChef Tutorial

RuleChef takes labeled examples and uses an LLM to write regex rules that do the same job. Once the rules are learned, they run locally — no LLM needed, no API costs, sub-millisecond per query.

**Why rules instead of calling an LLM every time?**
- **Free to run** — no API calls at inference time
- **Fast** — under 1ms per query
- **Predictable** — same input always gives the same output
- **Transparent** — you can read, edit, and debug every rule

In this tutorial, we'll extract medical entities from clinical prescription text (DRUG, DOSAGE, FREQUENCY, CONDITION) — a common NER task in healthcare NLP. We'll start with just 3 examples and explore every tool RuleChef offers, then scale up to a real benchmark.

## Part 1: Getting Started

In [ ]:
# Install (run once)
# !pip install rulechef datasets

In [ ]:
import os

# Set your API key — any OpenAI-compatible provider works (OpenAI, Groq, Together, etc.)
# os.environ["OPENAI_API_KEY"] = "your-key-here"

In [ ]:
import tempfile

from openai import OpenAI

from rulechef import RuleChef, RuleFormat, Task, TaskType

# --- LLM client ---
# Using Groq (free tier). Any OpenAI-compatible API works.
# For OpenAI: just OpenAI() with OPENAI_API_KEY set.
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
)
MODEL = "moonshot"

# --- Task definition ---
task = Task(
    name="Medical Prescription NER",
    description=(
        "Extract DRUG, DOSAGE, FREQUENCY, and CONDITION entities from clinical "
        "prescription text. DRUG = medication name, DOSAGE = amount/strength, "
        "FREQUENCY = how often taken, CONDITION = medical condition being treated."
    ),
    input_schema={"text": "str"},
    output_schema={
        "entities": "List[{text: str, start: int, end: int, type: DRUG|DOSAGE|FREQUENCY|CONDITION}]"
    },
    type=TaskType.NER,
    text_field="text",
)

# --- Create RuleChef ---
storage = tempfile.mkdtemp(prefix="rulechef_tutorial_")
chef = RuleChef(
    task,
    client,
    dataset_name="medical_ner_tutorial",
    storage_path=storage,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],  # regex-only for transparency
    use_grex=True,  # use grex to suggest regex patterns from examples
)
print(f"Storage: {storage}")

### Add examples

We start with just 3 examples — each showing a clinical prescription and its entity spans. Each entity has `text`, `start`/`end` character offsets, and a `type`.

Notice these aren't cookie-cutter — they're deliberately varied:
1. **Standard format** — `"Metformin 500mg twice daily for type 2 diabetes."`
2. **Verb prefix + lowercase drug** — `"Take lisinopril 10mg..."` — the drug isn't at position 0
3. **Filler text the rules must skip** — `"...every 6 hours as needed for pain."` — "as needed" sits between FREQUENCY and CONDITION

This variety means the LLM can't get away with one simple pattern. It has to write rules that handle positional differences.

In [ ]:
examples = [
    (
        "Metformin 500mg twice daily for type 2 diabetes.",
        [
            {"text": "Metformin", "start": 0, "end": 9, "type": "DRUG"},
            {"text": "500mg", "start": 10, "end": 15, "type": "DOSAGE"},
            {"text": "twice daily", "start": 16, "end": 27, "type": "FREQUENCY"},
            {"text": "type 2 diabetes", "start": 32, "end": 47, "type": "CONDITION"},
        ],
    ),
    (
        "Take lisinopril 10mg once daily for high blood pressure.",
        [
            {"text": "lisinopril", "start": 5, "end": 15, "type": "DRUG"},
            {"text": "10mg", "start": 16, "end": 20, "type": "DOSAGE"},
            {"text": "once daily", "start": 21, "end": 31, "type": "FREQUENCY"},
            {"text": "high blood pressure", "start": 36, "end": 55, "type": "CONDITION"},
        ],
    ),
    (
        "Ibuprofen 400mg every 6 hours as needed for pain.",
        [
            {"text": "Ibuprofen", "start": 0, "end": 9, "type": "DRUG"},
            {"text": "400mg", "start": 10, "end": 15, "type": "DOSAGE"},
            {"text": "every 6 hours", "start": 16, "end": 29, "type": "FREQUENCY"},
            {"text": "pain", "start": 44, "end": 48, "type": "CONDITION"},
        ],
    ),
]

for text, entities in examples:
    chef.add_example({"text": text}, {"entities": entities})

# Examples sit in a buffer until we call learn_rules()
print(chef.get_buffer_stats())

### Learn rules

This sends the examples to the LLM, which writes regex rules. Then it tests them against the examples and patches any mistakes.

In [4]:
rules, eval_result = chef.learn_rules()


📥 Converting 5 buffered examples to dataset...
   (0 corrections, 0 LLM, 5 human)
✓ Converted to dataset: 0 corrections, 5 examples

Learning rules from 5 training items
  Corrections: 0 (high value)
  Examples: 5
  Mode: Synthesis + Refinement (max 3 iterations)

📚 Per-class synthesis: 5 classes, up to 5 rules each
  [1/5] beneficiary_not_allowed: 2 rules (1.1s)
  [2/5] card_arrival: 5 rules (1.3s)
      Validation error: global flags not at the start of the expression at position 90
   ⚠ Skipped invalid rule: Disposable card limits keyword
  [3/5] disposable_card_limits: 2 rules (1.1s)
      Validation error: nothing to repeat at position 76
   ⚠ Skipped invalid rule: exchange_rate_question
  [4/5] exchange_rate: 4 rules (1.1s)
      Validation error: global flags not at the start of the expression at position 14
   ⚠ Skipped invalid rule: Pending Withdrawal Mention
      Validation error: global flags not at the start of the expression at position 12
   ⚠ Skipped invalid rule: Pend

## Part 2: Inspect Everything

### Rule summary

A quick overview of what was learned:

In [ ]:
for i, s in enumerate(chef.get_rules_summary(), 1):
    print(
        f"{i:2d}. {s['name']:<40s} format={s['format']}  priority={s['priority']}  confidence={s['confidence']}"
    )

### The actual regex patterns

The summary hides the regex. Let's look at the real rules — what regex matches, and what label it produces:

In [ ]:
for rule in chef.dataset.rules:
    entity_type = rule.output_template.get("type", "?") if rule.output_template else "?"
    print(f"[{rule.id}] {rule.name}")
    print(f"  Pattern:     {rule.content}")
    print(f"  Entity type: {entity_type}")
    print(f"  Priority:    {rule.priority}")
    print()

### Try them out

Let's test on texts the rules have **never seen**. Watch for where they break:
- The first should mostly work — same structure as training.
- `"Take aspirin..."` tests whether the rules learned the verb-prefix pattern from just one example.
- `"at bedtime"` is a frequency pattern that wasn't in any training example — will the regex generalize?
- The last sentence has no medical entities at all — it shouldn't match anything.

In [ ]:
test_texts = [
    "Metformin 1000mg once daily for insulin resistance.",  # should work
    "Take aspirin 81mg daily for cardiac prevention.",  # "Take" prefix
    "Gabapentin 300mg at bedtime for nerve pain.",  # "at bedtime" — not seen yet
    "The patient reported feeling dizzy after the procedure.",  # no entities
]

for text in test_texts:
    result = chef.extract({"text": text})
    entities = result.get("entities", [])
    print(f"Input: {text}")
    for e in entities:
        print(f"  [{e['type']}] {e['text']!r}")
    if not entities:
        print("  (no entities found)")
    print()

In [ ]:
# How fast is it? No LLM call — just regex matching.
%timeit chef.extract({"text": "Metformin 500mg twice daily for type 2 diabetes."})

### Evaluate

How well do the rules do on the training data? `evaluate()` runs every rule against every example and computes precision, recall, and F1 per class.

In [ ]:
eval_result = chef.evaluate()

### Per-rule metrics

Which rules are pulling their weight? `get_rule_metrics()` evaluates each rule in isolation — useful for finding dead rules (0 matches) or harmful ones (low precision).

In [ ]:
rule_metrics = chef.get_rule_metrics()

### Delete a rule

If a rule is dead (0 matches) or harmful, you can remove it:

In [ ]:
# Find a dead rule (0 matches) if any
dead = [m for m in rule_metrics if m.matches == 0]
if dead:
    print(f"Deleting dead rule: {dead[0].rule_name} (id={dead[0].rule_id})")
    chef.delete_rule(dead[0].rule_id)
    print(f"Rules remaining: {len(chef.dataset.rules)}")
else:
    print("No dead rules found — all rules match something.")

## Part 3: Improve the Rules

### Corrections

When the rules get something wrong, you can correct them. Corrections are the **highest-value training signal** — they show exactly where rules fail, paired with the right answer.

We'll test with `"Prescribe amoxicillin 250mg three times a day for strep throat."` — this is designed to stress the current rules in three ways:
- **Verb prefix** — `"Prescribe"` pushes the drug to position 10, not 0
- **Lowercase drug** — `"amoxicillin"` after a verb, unlike the capitalized drugs the rules saw
- **Unseen frequency** — `"three times a day"` vs the `"twice daily"` / `"once daily"` / `"every N hours"` the rules were trained on

Run the cell and see what the rules miss — then we'll correct it.

In [ ]:
# Try a new prescription — the rules might miss entities with unusual patterns
test_input = {"text": "Prescribe amoxicillin 250mg three times a day for strep throat."}
model_output = chef.extract(test_input)
print(f"Got: {model_output}")
print(
    "Expected: DRUG=amoxicillin, DOSAGE=250mg, FREQUENCY=three times a day, CONDITION=strep throat"
)

# Add a correction — tell RuleChef what the right answer is
chef.add_correction(
    test_input,
    model_output=model_output,
    expected_output={
        "entities": [
            {"text": "amoxicillin", "start": 10, "end": 21, "type": "DRUG"},
            {"text": "250mg", "start": 22, "end": 27, "type": "DOSAGE"},
            {"text": "three times a day", "start": 28, "end": 45, "type": "FREQUENCY"},
            {"text": "strep throat", "start": 50, "end": 62, "type": "CONDITION"},
        ]
    },
    feedback="DRUG can appear after a verb like 'Prescribe' and may be lowercase. "
    "FREQUENCY 'three times a day' is a valid variant of 'three times daily'.",
)

In [ ]:
# Incremental learn — patch existing rules instead of starting from scratch
chef.learn_rules(incremental_only=True)

In [ ]:
# Test again — did it fix it?
result = chef.extract({"text": "Prescribe amoxicillin 250mg three times a day for strep throat."})
entities = result.get("entities", [])
print("Entities found:")
for e in entities:
    print(f"  [{e['type']}] {e['text']!r}")

### Feedback

**Corrections** fix a specific wrong answer. **Feedback** is general guidance that steers *all future* rule generation — it's injected into the LLM prompt as "user guidance."

Use feedback when you know something the examples don't fully show — like "drugs can appear after verbs" or "frequency can be 'at bedtime'." This is cheaper than adding labeled examples for every edge case.

In [ ]:
chef.add_feedback(
    "DRUG names are medication names that may appear at the start of the text or after "
    "a verb like 'Take' or 'Prescribe' (in which case they may be lowercase). "
    "DOSAGE follows the drug name and contains a number with units (mg, mcg, units, %). "
    "FREQUENCY describes how often: 'once/twice/three times daily', 'three times a day', "
    "'every N hours', 'daily', 'at bedtime', 'each morning'. "
    "CONDITION follows 'for' and describes the medical condition being treated — "
    "it can be multi-word (e.g., 'type 2 diabetes', 'high blood pressure')."
)
print(f"Feedback count: {chef.get_stats()['feedback']}")

In [ ]:
# Re-learn — feedback gets injected into the synthesis prompt
chef.learn_rules()

### Sampling strategies

When you have lots of examples, RuleChef can't fit them all in the LLM prompt. It picks a subset using a **sampling strategy**:

| Strategy | What it does |
|----------|-------------|
| `balanced` | Takes examples in order (default) |
| `recent` | Newest examples first |
| `diversity` | Evenly spaces picks across the dataset |
| `uncertain` | Low-confidence examples first |
| `varied` | Mix of recent + diverse + uncertain |
| `corrections_first` | Corrections always go first (they already do by default, but this also sorts examples by recency) |

All strategies always include corrections first — they're the highest-value signal.

You can set a default strategy when creating RuleChef, or override per call:

In [ ]:
# Override sampling for a single learn call
chef.learn_rules(sampling_strategy="diversity")

## Part 4: Under the Hood

What actually goes to the LLM? Let's look at the prompts RuleChef builds.

### The synthesis prompt

This is the prompt that asks the LLM to generate rules from examples. It includes:
1. **Task description** — what we're trying to do
2. **Data evidence** — patterns found in the examples (including auto-generated regex hints from `grex`)
3. **Training examples** — the actual input/output pairs
4. **User feedback** — any guidance you've added
5. **Format instructions** — how to write regex rules, with technique guides
6. **Response schema** — the exact JSON format to return

In [ ]:
# Build the synthesis prompt (what the LLM sees)
prompt = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)
print(f"Prompt length: {len(prompt)} chars")
print("=" * 80)
print(prompt[:3000])
print("\n... (truncated) ...")

### The patch prompt

When rules don't match all examples, RuleChef runs a refinement loop. The patch prompt shows the LLM the current rules, the failures, and asks it to fix them:

In [ ]:
# Build a patch prompt — shows current rules + failures
# First, get some failures to show
from rulechef.evaluation import evaluate_dataset

eval_result = evaluate_dataset(chef.dataset.rules, chef.dataset, chef.learner._apply_rules)

if eval_result.failures:
    patch_prompt = chef.learner._build_patch_prompt(
        chef.dataset.rules, eval_result.failures, max_rules=5, dataset=chef.dataset
    )
    print(f"Patch prompt length: {len(patch_prompt)} chars")
    print("=" * 80)
    print(patch_prompt[:3000])
    print("\n... (truncated) ...")
else:
    print("No failures — all examples matched! No patch prompt needed.")

### grex: regex hints from examples

[grex](https://github.com/pemistahl/grex) is a library that infers regex patterns from example strings. You give it strings, it gives you a regex that matches all of them. RuleChef uses this to give the LLM concrete pattern suggestions during rule synthesis.

Let's first see what grex does on its own, then see how RuleChef uses it in prompts.

In [ ]:
import re

try:
    from grex import RegExpBuilder

    # Example 1: Dates — grex finds the structural pattern
    dates = ["2024-01-15", "2024-02-28", "2023-12-01", "2025-06-30"]
    exact = RegExpBuilder.from_test_cases(dates).without_anchors().build()
    generalized = (
        RegExpBuilder.from_test_cases(dates)
        .without_anchors()
        .with_conversion_of_digits()
        .with_conversion_of_repetitions()
        .build()
    )
    print("=== Dates ===")
    print(f"  Input:      {dates}")
    print(f"  Exact:      {exact}")
    print(f"  Generalized: {generalized}")
    # Verify the generalized pattern matches new dates
    print(f"  Matches '2026-03-14'? {bool(re.search(generalized, '2026-03-14'))}")
    print()

    # Example 2: Dollar amounts
    amounts = ["$10.99", "$5.00", "$249.95", "$1.50"]
    exact = RegExpBuilder.from_test_cases(amounts).without_anchors().build()
    generalized = (
        RegExpBuilder.from_test_cases(amounts)
        .without_anchors()
        .with_conversion_of_digits()
        .with_conversion_of_repetitions()
        .build()
    )
    print("=== Dollar amounts ===")
    print(f"  Input:      {amounts}")
    print(f"  Exact:      {exact}")
    print(f"  Generalized: {generalized}")
    print()

    # Example 3: Natural language — exact pattern is just alternation
    phrases = ["exchange rate", "current rates", "conversion rate"]
    exact = RegExpBuilder.from_test_cases(phrases).without_anchors().build()
    print("=== Natural language phrases ===")
    print(f"  Input:      {phrases}")
    print(f"  Exact:      {exact}")
    print("  (No structural pattern — text is too diverse to generalize)")

except ImportError:
    print("grex not installed. Install with: pip install rulechef[grex]")

In [ ]:
# Extract just the data evidence section from the prompt
prompt_with_grex = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)

# Find the data evidence section
start = prompt_with_grex.find("DATA EVIDENCE FROM TRAINING:")
end = prompt_with_grex.find("\n\n", start + 1) if start != -1 else -1
if start != -1:
    evidence_with = prompt_with_grex[start:end].strip()
    print("=== With grex (default) ===")
    print(evidence_with)
else:
    print("DATA EVIDENCE section not found — grex may not be installed.")
    print("Install with: pip install rulechef[grex]")

In [ ]:
# Now disable grex and compare
chef.learner.prompt_builder.use_grex = False
prompt_without_grex = chef.learner._build_synthesis_prompt(chef.dataset, max_rules=10)

start = prompt_without_grex.find("DATA EVIDENCE FROM TRAINING:")
end = prompt_without_grex.find("\n\n", start + 1) if start != -1 else -1
if start != -1:
    evidence_without = prompt_without_grex[start:end].strip()
    print("=== Without grex ===")
    print(evidence_without)
else:
    print("No DATA EVIDENCE section — this task type may not generate one.")

# Restore grex
chef.learner.prompt_builder.use_grex = True

# Show the difference
print("\n" + "=" * 60)
print(f"Prompt length WITH grex:    {len(prompt_with_grex):,} chars")
print(f"Prompt length WITHOUT grex: {len(prompt_without_grex):,} chars")
print(f"grex adds {len(prompt_with_grex) - len(prompt_without_grex):,} chars of regex hints")

## Part 5: Agentic Learning

So far the learning loop uses simple heuristics: "did F1 improve? keep going." The **Agentic Coordinator** replaces those heuristics with an LLM that reads the per-class metrics after each iteration and writes targeted guidance: *"FREQUENCY recall is 60% — the rules miss 'at bedtime' and 'each morning', add alternation patterns for these."*

It also decides when to **stop early** — no point iterating if everything is at 100%.

To give it a real workout, we'll add 9 more varied examples: different dosage units (`mcg`, `units`), new frequency patterns (`"at bedtime"`, `"each morning"`, `"three times daily"`), filler text (`"before breakfast"`, `"taper over 2 weeks"`), and a multi-word drug (`"Insulin glargine"`).

In [ ]:
from rulechef.coordinator import AgenticCoordinator

# Add more examples from the full dataset so the agentic coordinator has more to work with
more_examples = [
    (
        "Aspirin 81mg daily for cardiac prevention.",
        [
            {"text": "Aspirin", "start": 0, "end": 7, "type": "DRUG"},
            {"text": "81mg", "start": 8, "end": 12, "type": "DOSAGE"},
            {"text": "daily", "start": 13, "end": 18, "type": "FREQUENCY"},
            {"text": "cardiac prevention", "start": 23, "end": 41, "type": "CONDITION"},
        ],
    ),
    (
        "Omeprazole 20mg once daily before breakfast for acid reflux.",
        [
            {"text": "Omeprazole", "start": 0, "end": 10, "type": "DRUG"},
            {"text": "20mg", "start": 11, "end": 15, "type": "DOSAGE"},
            {"text": "once daily", "start": 16, "end": 26, "type": "FREQUENCY"},
            {"text": "acid reflux", "start": 48, "end": 59, "type": "CONDITION"},
        ],
    ),
    (
        "Atorvastatin 40mg at bedtime for high cholesterol.",
        [
            {"text": "Atorvastatin", "start": 0, "end": 12, "type": "DRUG"},
            {"text": "40mg", "start": 13, "end": 17, "type": "DOSAGE"},
            {"text": "at bedtime", "start": 18, "end": 28, "type": "FREQUENCY"},
            {"text": "high cholesterol", "start": 33, "end": 49, "type": "CONDITION"},
        ],
    ),
    (
        "Prednisone 60mg once daily for asthma, taper over 2 weeks.",
        [
            {"text": "Prednisone", "start": 0, "end": 10, "type": "DRUG"},
            {"text": "60mg", "start": 11, "end": 15, "type": "DOSAGE"},
            {"text": "once daily", "start": 16, "end": 26, "type": "FREQUENCY"},
            {"text": "asthma", "start": 31, "end": 37, "type": "CONDITION"},
        ],
    ),
    (
        "Warfarin 5mg once daily for deep vein thrombosis.",
        [
            {"text": "Warfarin", "start": 0, "end": 8, "type": "DRUG"},
            {"text": "5mg", "start": 9, "end": 12, "type": "DOSAGE"},
            {"text": "once daily", "start": 13, "end": 23, "type": "FREQUENCY"},
            {"text": "deep vein thrombosis", "start": 28, "end": 48, "type": "CONDITION"},
        ],
    ),
    (
        "Sertraline 50mg once daily for generalized anxiety disorder.",
        [
            {"text": "Sertraline", "start": 0, "end": 10, "type": "DRUG"},
            {"text": "50mg", "start": 11, "end": 15, "type": "DOSAGE"},
            {"text": "once daily", "start": 16, "end": 26, "type": "FREQUENCY"},
            {"text": "generalized anxiety disorder", "start": 31, "end": 59, "type": "CONDITION"},
        ],
    ),
    (
        "Levothyroxine 100mcg each morning for hypothyroidism.",
        [
            {"text": "Levothyroxine", "start": 0, "end": 13, "type": "DRUG"},
            {"text": "100mcg", "start": 14, "end": 20, "type": "DOSAGE"},
            {"text": "each morning", "start": 21, "end": 33, "type": "FREQUENCY"},
            {"text": "hypothyroidism", "start": 38, "end": 52, "type": "CONDITION"},
        ],
    ),
    (
        "Insulin glargine 18 units at bedtime for diabetes mellitus.",
        [
            {"text": "Insulin glargine", "start": 0, "end": 16, "type": "DRUG"},
            {"text": "18 units", "start": 17, "end": 25, "type": "DOSAGE"},
            {"text": "at bedtime", "start": 26, "end": 36, "type": "FREQUENCY"},
            {"text": "diabetes mellitus", "start": 41, "end": 58, "type": "CONDITION"},
        ],
    ),
    (
        "Gabapentin 300mg three times daily for neuropathic pain.",
        [
            {"text": "Gabapentin", "start": 0, "end": 10, "type": "DRUG"},
            {"text": "300mg", "start": 11, "end": 16, "type": "DOSAGE"},
            {"text": "three times daily", "start": 17, "end": 34, "type": "FREQUENCY"},
            {"text": "neuropathic pain", "start": 39, "end": 55, "type": "CONDITION"},
        ],
    ),
]

for text, entities in more_examples:
    chef.add_example({"text": text}, {"entities": entities})

print(f"Buffer: {chef.get_buffer_stats()['new_examples']} new examples")

In [ ]:
# Switch to agentic coordinator
chef.coordinator = AgenticCoordinator(client, model=MODEL)

# Learn with more refinement iterations — watch the coordinator's guidance
# For NER, it will analyze per-entity-type precision/recall and guide synthesis
rules, eval_result = chef.learn_rules(max_refinement_iterations=5)

In [ ]:
# Check the results
chef.evaluate()

### Rule pruning

After multiple refinement iterations, the ruleset can get bloated with redundant rules. Enable `prune_after_learn` and the coordinator will merge similar patterns and remove pure noise — with a safety net that reverts if quality drops.

In [ ]:
print(f"Rules before pruning: {len(chef.dataset.rules)}")

# Enable pruning
chef.coordinator = AgenticCoordinator(client, model=MODEL, prune_after_learn=True)

rules, eval_result = chef.learn_rules(max_refinement_iterations=3)

print(f"\nRules after pruning: {len(chef.dataset.rules)}")

## Part 6: LLM Observability

What if you already have an LLM extracting entities in production? Instead of defining a task and manually labeling examples, you can **wrap your existing OpenAI client** and RuleChef will:

1. **Observe** every LLM call — capturing messages and responses
2. **Auto-discover** the task schema from the patterns it sees
3. **Map** observations to the discovered schema
4. **Learn rules** that replicate the LLM's behavior

```
Your code → wrapped_client.chat.completions.create(...) → LLM responds
                         ↓ (observed)
                   RuleChef captures raw interactions
                         ↓ (at learn time)
                   Discover task → Map to schema → Learn rules
```

**Zero-config**: no Task definition needed, no custom extractors. Just wrap and go.

In [ ]:
# Zero-config: observe an LLM extracting entities from prescriptions
storage_obs = tempfile.mkdtemp(prefix="rulechef_observe_")
observer_chef = RuleChef(
    client=client,  # No task!
    dataset_name="medical_observed",
    storage_path=storage_obs,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
    use_grex=True,
)

# Wrap the client — returns the same client object, now observed
wrapped_client = observer_chef.start_observing(client, auto_learn=False)
print("Observing... make some LLM calls.")

In [ ]:
import json

# Simulate your existing LLM pipeline — extract entities from prescriptions
# RuleChef silently captures every call (messages + response).
prescriptions = [
    "Metformin 500mg twice daily for type 2 diabetes.",
    "Take lisinopril 10mg once daily for high blood pressure.",
    "Ibuprofen 400mg every 6 hours as needed for pain.",
    "Aspirin 81mg daily for cardiac prevention.",
    "Omeprazole 20mg once daily before breakfast for acid reflux.",
    "Atorvastatin 40mg at bedtime for high cholesterol.",
    "Prednisone 60mg once daily for asthma, taper over 2 weeks.",
    "Warfarin 5mg once daily for deep vein thrombosis.",
]

system_prompt = (
    "Extract medical entities from the prescription text. "
    "Return JSON with an 'entities' list. Each entity has: "
    "'text' (exact span), 'start' (char offset), 'end' (char offset), "
    "and 'type' (one of: DRUG, DOSAGE, FREQUENCY, CONDITION). "
    "Return only valid JSON, no explanation."
)

for prescription in prescriptions:
    response = wrapped_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prescription},
        ],
        temperature=0,
    )
    result = response.choices[0].message.content.strip()
    try:
        entities = json.loads(result).get("entities", [])
        types = ", ".join(e.get("type", "?") for e in entities)
        print(f"{prescription[:55]:57s} → [{types}]")
    except Exception:
        print(f"{prescription[:55]:57s} → (parse error)")

# Check what was captured
print(f"\nObserver stats: {observer_chef.get_buffer_stats().get('observer', {})}")

In [ ]:
# learn_rules() does everything:
#   1. Discovers the task schema from raw observations (since no task was provided)
#   2. Maps each observation to the discovered schema
#   3. Commits mapped examples to buffer → dataset
#   4. Synthesizes + refines rules
rules, eval_result = observer_chef.learn_rules()

# Show what was discovered
print(f"\nDiscovered task: {observer_chef.task.name}")
print(f"  Type: {observer_chef.task.type.value}")
print(f"  Input schema: {observer_chef.task.input_schema}")
print(f"  Output schema: {observer_chef.task.output_schema}")

# Stop observing — restores the original client
observer_chef.stop_observing()

# The rules work without any LLM calls
for text in [
    "Warfarin 5mg once daily for deep vein thrombosis.",
    "Tramadol 50mg every 6 hours for chronic back pain.",
]:
    result = observer_chef.extract({"text": text})
    entities = result.get("entities", [])
    print(f"\n{text}")
    for e in entities:
        print(f"  [{e.get('type', '?')}] {e.get('text', '?')!r}")
    if not entities:
        print("  (no entities)")

## Part 7: Scale Up

Everything above used NER on hand-crafted prescriptions. But does RuleChef work on real, messy data — and on a different task type?

[Banking77](https://huggingface.co/datasets/legacy-datasets/banking77) has 13,000 banking customer queries across 77 intent classes. We'll pick 5 classes, give RuleChef just **5 examples per class** (25 total), and test on held-out data it's never seen. This is a **classification** task — `TaskType.CLASSIFICATION` instead of `TaskType.NER` — so the same API, different task type.

In [ ]:
# Banking77 task definition (classification, separate from the NER task above)
CLASSES = [
    "exchange_rate",
    "card_arrival",
    "beneficiary_not_allowed",
    "disposable_card_limits",
    "pending_cash_withdrawal",
]

banking_task = Task(
    name="Banking Intent Classification",
    description=(
        f"Classify banking customer queries into one of these intents: {', '.join(CLASSES)}"
    ),
    input_schema={"text": "str"},
    output_schema={"label": "str"},
    type=TaskType.CLASSIFICATION,
    text_field="text",
)

In [ ]:
import random
import uuid
from collections import defaultdict

from datasets import load_dataset

from rulechef.core import Dataset, Example

# Load Banking77
ds = load_dataset("legacy-datasets/banking77")
label_names = ds["train"].features["label"].names

train_records = [{"text": row["text"], "label": label_names[row["label"]]} for row in ds["train"]]
test_records = [{"text": row["text"], "label": label_names[row["label"]]} for row in ds["test"]]

print(
    f"Banking77: {len(train_records)} train, {len(test_records)} test, {len(label_names)} classes"
)

In [ ]:
# Sample 5-shot per class for our 5 classes
rng = random.Random(42)
by_label = defaultdict(list)
for ex in train_records:
    if ex["label"] in CLASSES:
        by_label[ex["label"]].append(ex)

train_sample = []
train_remaining = []
for label in sorted(CLASSES):
    exs = by_label[label]
    rng.shuffle(exs)
    train_sample.extend(exs[:5])
    train_remaining.extend(exs[5:])

# Filter test set to our 5 classes
test_data = [ex for ex in test_records if ex["label"] in CLASSES]

print(f"Training: {len(train_sample)} examples (5-shot x {len(CLASSES)} classes)")
print(f"Held-out eval: {len(train_remaining)} examples")
print(f"Held-out test: {len(test_data)} examples")

# Show a few
for ex in train_sample[:3]:
    print(f"  {ex['label']:30s} | {ex['text']}")

In [ ]:
# Create a fresh chef for the scale-up
storage2 = tempfile.mkdtemp(prefix="rulechef_banking77_")
chef2 = RuleChef(
    banking_task,
    client,
    dataset_name="banking77_5shot",
    storage_path=storage2,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
    use_grex=True,
)

# Add all 25 training examples
for ex in train_sample:
    chef2.add_example({"text": ex["text"]}, {"label": ex["label"]})

# Learn with evaluation
rules, eval_result = chef2.learn_rules(max_refinement_iterations=3)

### Evaluate on held-out test data

The rules were learned from 25 examples. Let's see how they do on data they've never seen:

In [ ]:
import time

# Build a test Dataset
test_dataset = Dataset(name="banking77_test", task=banking_task)
for ex in test_data:
    test_dataset.examples.append(
        Example(
            id=str(uuid.uuid4())[:8],
            input={"text": ex["text"]},
            expected_output={"label": ex["label"]},
            source="benchmark",
        )
    )

# Evaluate
t0 = time.time()
test_eval = evaluate_dataset(chef2.dataset.rules, test_dataset, chef2.learner._apply_rules)
elapsed = time.time() - t0

print(f"\nHeld-out test results ({len(test_data)} examples):")
print(f"  Accuracy:    {test_eval.exact_match:.1%}")
print(f"  Precision:   {test_eval.micro_precision:.1%}")
print(f"  Recall:      {test_eval.micro_recall:.1%}")
print(f"  Micro F1:    {test_eval.micro_f1:.1%}")
print(f"  Macro F1:    {test_eval.macro_f1:.1%}")
print(f"  Per query:   {elapsed / len(test_data) * 1000:.2f}ms")
print(f"  Rules:       {len(chef2.dataset.rules)}")

print("\nPer class:")
for cm in sorted(test_eval.per_class, key=lambda c: c.f1, reverse=True):
    print(
        f"  {cm.label:35s} F1={cm.f1:.0%}  P={cm.precision:.0%}  R={cm.recall:.0%}  (TP={cm.tp} FP={cm.fp} FN={cm.fn})"
    )

### Persistence

Rules are automatically saved to disk as JSON. You can reload them by creating a new `RuleChef` with the same `dataset_name` and `storage_path`.

In [ ]:
import json
from pathlib import Path

# Show the saved file
saved_file = Path(storage2) / "banking77_5shot.json"
data = json.loads(saved_file.read_text())
print(f"Saved to: {saved_file}")
print(f"File size: {saved_file.stat().st_size / 1024:.1f} KB")
print(f"Contains: {len(data.get('rules', []))} rules, {len(data.get('examples', []))} examples")

In [ ]:
# Recreate chef from disk — rules load automatically
chef3 = RuleChef(
    banking_task,
    client,
    dataset_name="banking77_5shot",
    storage_path=storage2,
    model=MODEL,
    allowed_formats=[RuleFormat.REGEX],
)
print(f"Loaded {len(chef3.dataset.rules)} rules from disk")

# Works immediately
result = chef3.extract({"text": "what's the exchange rate for dollars?"})
print(f"Extract: {result}")

# Measure throughput
t0 = time.time()
for ex in test_data[:100]:
    chef3.extract({"text": ex["text"]})
elapsed = time.time() - t0
print(f"\n100 queries in {elapsed * 1000:.1f}ms ({elapsed / 100 * 1000:.2f}ms per query)")

## Summary

In this tutorial you learned how to use RuleChef on a medical NER task (extracting DRUG, DOSAGE, FREQUENCY, and CONDITION entities from prescription text), then benchmarked it on a classification task to show it works across task types.

| # | What you learned | Key API |
|---|-----------------|---------|
| 1 | Define a task — describe entity types and schema | `Task(...)` |
| 2 | Add examples — input text + labeled entity spans | `chef.add_example()` |
| 3 | Learn rules — the LLM writes regex rules from examples | `chef.learn_rules()` |
| 4 | Inspect rules — see patterns, entity types, priorities | `chef.get_rules_summary()`, `chef.dataset.rules` |
| 5 | Extract — run NER with sub-ms latency, no LLM needed | `chef.extract()` |
| 6 | Evaluate — precision/recall/F1 per entity type and per rule | `chef.evaluate()`, `chef.get_rule_metrics()` |
| 7 | Correct mistakes — add corrections and re-learn incrementally | `chef.add_correction()` |
| 8 | Give feedback — steer rule generation with natural language | `chef.add_feedback()` |
| 9 | Sampling strategies — control which examples the LLM sees | `sampling_strategy="diversity"` |
| 10 | Under the hood — inspect synthesis and patch prompts, grex hints | `learner._build_synthesis_prompt()` |
| 11 | Agentic learning — LLM-guided refinement per entity type | `AgenticCoordinator` |
| 12 | Rule pruning — merge redundant rules with safety net | `prune_after_learn=True` |
| 13 | LLM observability — wrap your client, auto-discover task, learn rules | `chef.start_observing()` |
| 14 | Scale up — from 3 examples to a real benchmark dataset | `Banking77` |
| 15 | Persistence — rules save to disk and reload automatically | `storage_path=` |

**Docs & further reading:**
- [Full documentation](https://krlabsorg.github.io/rulechef) — installation, guides, API reference
- [Task types](https://krlabsorg.github.io/rulechef/guide/task-types/) — NER, extraction, classification, transformation
- [Learning & refinement](https://krlabsorg.github.io/rulechef/guide/learning/) — buffer architecture, sampling, incremental patching
- [Evaluation & feedback](https://krlabsorg.github.io/rulechef/guide/evaluation/) — metrics, corrections, custom matchers
- [Advanced features](https://krlabsorg.github.io/rulechef/guide/advanced/) — observation mode, spaCy patterns, training data logger
- [GitHub](https://github.com/KRLabsOrg/rulechef)

**What to try next:**
- Add more examples from your own data
- Use corrections when the extractor misses entities
- Try `RuleFormat.CODE` for more complex matching logic (e.g., drug name lists)
- Try `RuleFormat.SPACY` for dependency-aware patterns
- Use the CLI for interactive exploration: `rulechef`